In [7]:
import torchvision.models as models
import torch
import torch.nn as nn

In [12]:
class VGGFeatureExtractor(nn.Module):
    def __init__(self):
        super(VGGFeatureExtractor, self).__init__()
        
        vgg = models.vgg16(pretrained=True).features.eval()
        
        self.selected_layers = [ 0,  1, # block1_conv1 + relu
                                 5,  6, # block2_conv1 + relu
                                10, 11, # block3_conv1 + relu
                                17, 18, # block4_conv1 + relu
                                24, 25] # block5_conv1 + relu
        
        self.layers = nn.ModuleList([vgg[idx] for idx in self.selected_layers])
        
        for param in self.layers.parameters():
            param.requires_grad = False
            
    def forward(self, x):
        outputs = {}
        for i, layer in zip(self.selected_layers, self.layers):
            x = layer(x)
            outputs[f"layer_{i}"] = x
        return outputs

In [13]:
loss_model = VGGFeatureExtractor()

In [15]:
def print_model_info(model, input_size=(1, 3, 224, 224)):
    """ PyTorch 모델의 모든 Layer 정보를 출력 """
    
    print("--------------------------------------------------------------------------------")
    print(f"{'Layer Name':<30}{'Type':<30}{'Output Shape':<30}{'Activation':<20}{'Trainable'}")
    print("="*110)

    # 가상의 입력을 생성하여 Forward 수행
    x = torch.randn(*input_size).to(next(model.parameters()).device)

    for name, layer in model.named_children():
        x = layer(x)  # Forward pass
        
        # ✅ 활성화 함수(Activation) 확인
        activation = "None"
        if isinstance(layer, nn.ReLU):
            activation = "ReLU"
        elif isinstance(layer, nn.Sigmoid):
            activation = "Sigmoid"
        elif isinstance(layer, nn.Tanh):
            activation = "Tanh"

        # ✅ Layer 정보 출력
        print(f"{name:<30}{layer.__class__.__name__:<30}{str(tuple(x.shape)):<30}{activation:<20}{layer.requires_grad}")

    print("="*110)

In [25]:
def print_model_info(model, input_size=(1, 3, 256, 256)):
    """ PyTorch 모델의 모든 Layer 정보를 출력 (ModuleList 지원) """
    
    print("--------------------------------------------------------------------------------")
    print(f"{'Layer Name':<30}{'Type':<30}{'Output Shape':<30}{'Activation':<20}{'Trainable'}")
    print("="*110)

    # 가상의 입력을 생성하여 Forward 수행
    x = torch.randn(*input_size).to(next(model.parameters()).device)

    for name, layer in model.named_children():
        # ✅ ModuleList인 경우 개별 Layer 순회
        if isinstance(layer, nn.ModuleList):
            for sub_idx, sub_layer in enumerate(layer):
                x = sub_layer(x)  # ✅ 개별 Layer 실행
                activation = get_activation(sub_layer)
                print(f"{f'{name}[{sub_idx}] ({sub_layer.__class__.__name__})':<30}"
                      f"{str(tuple(x.shape)):<30}"
                      f"{activation:<20}")
        else:
            x = layer(x)  # ✅ 일반 Layer 실행
            activation = get_activation(layer)
            print(f"{f'{name} ({layer.__class__.__name__})':<30}"
                  f"{str(tuple(x.shape)):<30}"
                  f"{activation:<20}")

    print("="*110)

def get_activation(layer):
    """ Layer에서 Activation 함수 찾기 """
    if isinstance(layer, nn.ReLU):
        return "ReLU"
    elif isinstance(layer, nn.Sigmoid):
        return "Sigmoid"
    elif isinstance(layer, nn.Tanh):
        return "Tanh"
    return "None"

In [26]:
print_model_info(loss_model)

--------------------------------------------------------------------------------
Layer Name                    Type                          Output Shape                  Activation          Trainable
layers[0] (Conv2d)            (1, 64, 256, 256)             None                
layers[1] (ReLU)              (1, 64, 256, 256)             ReLU                
layers[2] (Conv2d)            (1, 128, 256, 256)            None                
layers[3] (ReLU)              (1, 128, 256, 256)            ReLU                
layers[4] (Conv2d)            (1, 256, 256, 256)            None                
layers[5] (ReLU)              (1, 256, 256, 256)            ReLU                
layers[6] (Conv2d)            (1, 512, 256, 256)            None                
layers[7] (ReLU)              (1, 512, 256, 256)            ReLU                
layers[8] (Conv2d)            (1, 512, 256, 256)            None                
layers[9] (ReLU)              (1, 512, 256, 256)            ReLU      

In [28]:

vgg_full = models.vgg16(weights='DEFAULT').features.eval()

In [29]:
print_model_info(vgg_full)

--------------------------------------------------------------------------------
Layer Name                    Type                          Output Shape                  Activation          Trainable
0 (Conv2d)                    (1, 64, 256, 256)             None                
1 (ReLU)                      (1, 64, 256, 256)             ReLU                
2 (Conv2d)                    (1, 64, 256, 256)             None                
3 (ReLU)                      (1, 64, 256, 256)             ReLU                
4 (MaxPool2d)                 (1, 64, 128, 128)             None                
5 (Conv2d)                    (1, 128, 128, 128)            None                
6 (ReLU)                      (1, 128, 128, 128)            ReLU                
7 (Conv2d)                    (1, 128, 128, 128)            None                
8 (ReLU)                      (1, 128, 128, 128)            ReLU                
9 (MaxPool2d)                 (1, 128, 64, 64)              None      

In [38]:
class VGGFeatureExtractor(nn.Module):
    def __init__(self):
        super(VGGFeatureExtractor, self).__init__()
        
        vgg = models.vgg16(weights='DEFAULT').features.eval()
        self.layers = nn.ModuleList(vgg[:25+1])
        for param in self.layers.parameters():
            param.requires_grad = False
            
    def forward(self, x):
        outputs = {}
        for i, layer in enumerate(self.layers):
            x = layer(x)
            outputs[f"layer_{i}"] = x
        return outputs

In [39]:
loss_model_new = VGGFeatureExtractor()
print_model_info(loss_model_new)

--------------------------------------------------------------------------------
Layer Name                    Type                          Output Shape                  Activation          Trainable
layers[0] (Conv2d)            (1, 64, 256, 256)             None                
layers[1] (ReLU)              (1, 64, 256, 256)             ReLU                
layers[2] (Conv2d)            (1, 64, 256, 256)             None                
layers[3] (ReLU)              (1, 64, 256, 256)             ReLU                
layers[4] (MaxPool2d)         (1, 64, 128, 128)             None                
layers[5] (Conv2d)            (1, 128, 128, 128)            None                
layers[6] (ReLU)              (1, 128, 128, 128)            ReLU                
layers[7] (Conv2d)            (1, 128, 128, 128)            None                
layers[8] (ReLU)              (1, 128, 128, 128)            ReLU                
layers[9] (MaxPool2d)         (1, 128, 64, 64)              None      